## CLIPSPY hands on tutorial

In [79]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

### Data types

CLIPS provides eight primitive data types for representing information. These types are float, integer, symbol, string, external-address, fact-address, instance-name and instance-address. Numeric information can be represented using floats and integers. Symbolic information can be represented using symbols and strings.

A number consists only of digits (0-9), a decimal point (.), a sign (+ or -), and, optionally, an (e) for exponential notation with its corresponding sign. A number is either stored as a float or an integer. Any number consisting of an optional sign followed by only digits is stored as an integer (represented internally by CLIPS as a C long integer). All other numbers are stored as floats (represented internally by CLIPS as a C double-precision float).

In [2]:
## Example floats and integers
env.eval("(+ 3 237e3 -100 0.001)")

236903.001

Symbols are any sequence of characters that starts with any printable ASCII character and is followed by zero or more printable ASCII characters. When a delimiter is found, the symbol is ended. The following characters act as delimiters: any non-printable ASCII character (including spaces, tabs, carriage returns, and line feeds), a double quote, opening and closing parentheses “(” and “)”, an ampersand “&”, a vertical bar “|”, a less than “<”, and a tilde “~”. A semicolon “;” starts a CLIPS comment and also acts as a delimiter.

In [3]:
## Example symbols
env.eval("foo")
env.eval("Hello")
env.eval("B76-HI")
env.eval("2-each")
env.eval("127A")

'127A'

A string is a set of characters that starts with a double quote (") and is followed by zero or more printable characters. A string ends with double quotes. Double quotes may be embedded within a string by placing a backslash (\) in front of the character. A backslash may be embedded by placing two consecutive backslash characters in the string.

In [4]:
## Example strings 
env.eval('"foo"')
env.eval('"a and b"')
env.eval('"1 number"')
env.eval('"a \\"quote\\""')

'a "quote"'

A multifield value is a sequence of zero or more single field values. When displayed by CLIPS, multifield values are enclosed in parentheses. Collectively, single and multifield values are referred to as values.

In [5]:
## Example multifield value
env.eval('(create$ x 3.0 "red" 567)')


['x', 3.0, 'red', 567]

A fact is a list of atomic values that are either referenced positionally (ordered facts) or by name (non-ordered or template facts). Ordered facts looks similar to multifields but are used directly for inference (see below). 

In [6]:
## Example ordered facts
env.assert_string("(the pump is on)")

ImpliedFact: f-1     (the pump is on)

In [7]:
env.assert_string("(grocery-list bread milk eggs)")

ImpliedFact: f-2     (grocery-list bread milk eggs)

Non-ordered facts allow for more structure to be used for representing knowledge. Non-ordered facts can be created using deftemplate or defclass. We will only use deftemplate in this class. 

In [8]:
# Simple template for a person
s = """
 (deftemplate person
    (slot name)
    (slot gender)
    (slot job)
    (slot location)
    (multislot interests))
"""
env.build(s)


In [9]:
# creates an instance of a 'person' fact 

template = env.find_template('person')

new_fact = template.new_fact()
new_fact['name'] = clips.Symbol('Richard')
new_fact['gender'] = clips.Symbol('Male')
new_fact['job'] = clips.Symbol('Professor')
new_fact['location'] = clips.Symbol('Pittsburgh')
new_fact['interests'] = [clips.Symbol('clinical decision support'), clips.Symbol('pharmacovigilance')]
new_fact.assertit()

for fact in env.facts():
    print(fact)

(initial-fact)
(the pump is on)
(grocery-list bread milk eggs)
(person (name Richard) (gender Male) (job Professor) (location Pittsburgh) (interests clinical decision support pharmacovigilance))


In [10]:
# a simpler way to construct a non-ordered fact but does not do any checking for 
# appropriate slot values 
new_fact = env.assert_string("""
 (person 
   (name Harry)
   (gender Male) 
   (job Professor) 
   (location Pittsburgh) 
   (interests human_computer_interaction Education))
)
""")

for fact in env.facts():
    print(fact)

(initial-fact)
(the pump is on)
(grocery-list bread milk eggs)
(person (name Richard) (gender Male) (job Professor) (location Pittsburgh) (interests clinical decision support pharmacovigilance))
(person (name Harry) (gender Male) (job Professor) (location Pittsburgh) (interests human_computer_interaction Education))


## Functions

Clips has many built in functions and they all look like lists

In [11]:
env.eval("(create$ apple banana peach)")


['apple', 'banana', 'peach']

In [12]:
env.eval("(first$ (create$ a b c))")

['a']

In [13]:
env.eval('(eq 1 1)')

'TRUE'

In [14]:
env.eval('(< 1 2)')

'TRUE'

Functions are evaluated from the inside out

In [15]:
env.eval('(< (+ 1 1)(+ 2 2))')

'TRUE'

In [16]:
env.eval('(eq 4 (+ 2 2))')

'TRUE'

Programmers can define custom functions. The simplest way to do this with clipspy is to write a python function and then use define_function to create the function in the CLIPS environment:

In [17]:
def testFunc(arg):
    print("I am within a Python function, argument: %f" % arg)
    return arg

env.define_function(testFunc)
ret = env.eval('(python-function testFunc 42.2)')
print("Eval returned %f" % ret)

I am within a Python function, argument: 42.200000
Eval returned 42.200000


Programmers can also use CLIPS deffunction to directly create functions 

In [18]:
env.build('''
  (deffunction concat-args (?a ?b ?c)
     (str-cat ?a ", " ?b ", and " ?c))
''')

In [19]:
env.eval('(concat-args One Two Three))')

'One, Two, and Three'

### Rules

Rules are defined using the defrule construct. 

Redefining a currently existing defrule causes the previous defrule with the same name to be removed even if the new definition has errors in it. 

The left-hand side (LHS) is made up of a series of conditional elements (CEs) that typically consist of pattern conditional elements (or just simply patterns) to be matched against pattern entities. An implicit and conditional element always surrounds all the patterns on the LHS. 

The right-hand side (RHS) contains a list of actions to be performed when the LHS of the rule is satisfied. In addition, the LHS of a rule may also contain declarations about the rule’s properties immediately following the rule’s name and comment. 

The arrow (=>) separates the LHS from the RHS. There is no limit to the number of conditional elements or actions a rule may have (other than the limitation placed by actual available memory). 

Actions are performed sequentially if, and only if, all conditional elements on the LHS are satisfied. If no conditional elements are on the LHS, the rule will automatically be activated. If no actions are on the RHS, the rule can be activated and fired but nothing will happen.

In [20]:
## Example
env.clear()
env.reset()

env.build("""
(deftemplate oav
 (slot object)
 (slot attribute)
 (slot value))
""")

env.build("""
(defrule example-rule
  "This is an example of a simple rule"
  (oav (object refrigerator)
       (attribute light)
       (value on))
  (oav (object refrigerator)
       (attribute door)
       (value open))
  =>
  (assert (oav (object refrigerator)
               (attribute food)
               (value spoiled))))
""")

env.assert_string("""
(oav (object refrigerator)
     (attribute light)
     (value on))
""")

env.assert_string("""
(oav (object refrigerator)
     (attribute door)
     (value open)))
""")

TemplateFact: f-2     (oav (object refrigerator) (attribute door) (value open))

In [21]:
for fact in env.facts():
    print(fact)

(initial-fact)
(oav (object refrigerator) (attribute light) (value on))
(oav (object refrigerator) (attribute door) (value open))


In [22]:
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(oav (object refrigerator) (attribute light) (value on))
(oav (object refrigerator) (attribute door) (value open))
(oav (object refrigerator) (attribute food) (value spoiled))


Another example of simple rules that uses the deffacts construct to assert facts in batch 

In [23]:
env.build("""
(deftemplate bed
    (slot unit)
    (slot bed-number)
    (slot patient-name))
""")

env.build("""
(deftemplate patient
    (slot patient-name)
    (multislot health-status))
""")

In [24]:
env.build("""
(deffacts StartStateOnSlides
    (bed (unit ER)(patient-name Ann))
    (bed (unit ICU)(bed-number 1)(patient-name Barry))
    (bed (unit ICU)(bed-number 2)(patient-name Cleo))
    (bed (unit Unit1)(bed-number 1))
    (bed (unit Unit1)(bed-number 2)(patient-name Darren))
    (bed (unit Unit2)(bed-number 1)(patient-name Eva))
    (bed (unit Unit2)(bed-number 2)(patient-name Frank))
    (patient (patient-name Ann)(health-status very-ill))
    (patient (patient-name Barry)(health-status not-very-ill infectious))
    (patient (patient-name Cleo)(health-status infectious very-ill))
    (patient (patient-name Darren)(health-status not-very-ill))
    (patient (patient-name Eva)(health-status infectious not-very-ill))
    (patient (patient-name Frank)(health-status free2go))
    )
""")

In [25]:
env.reset()
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')
for fact in env.facts():
    print(fact)

(initial-fact)
(bed (unit ER) (bed-number nil) (patient-name Ann))
(bed (unit ICU) (bed-number 1) (patient-name Barry))
(bed (unit ICU) (bed-number 2) (patient-name Cleo))
(bed (unit Unit1) (bed-number 1) (patient-name nil))
(bed (unit Unit1) (bed-number 2) (patient-name Darren))
(bed (unit Unit2) (bed-number 1) (patient-name Eva))
(bed (unit Unit2) (bed-number 2) (patient-name Frank))
(patient (patient-name Ann) (health-status very-ill))
(patient (patient-name Barry) (health-status not-very-ill infectious))
f-10    (patient (patient-name Cleo) (health-status infectious very-ill))
f-11    (patient (patient-name Darren) (health-status not-very-ill))
f-12    (patient (patient-name Eva) (health-status infectious not-very-ill))
f-13    (patient (patient-name Frank) (health-status free2go))


In [26]:
env.build("""
(defrule R1 "IF ER patient very ill and ICU bed free THEN admit to ICU bed"
    ?patient-fact <- (patient (patient-name ?ptname)(health-status $? very-ill $?))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ER))
    ?newbed-fact <- (bed (patient-name nil)(unit ICU)(bed-number ?newbedno))
    =>
    (modify ?newbed-fact (patient-name ?ptname))
    (retract ?oldbed-fact)
    (bind ?*log* 
        (str-cat ?*log* 
           (format nil "R1 Fired. Patient %s has been moved to ICU bed %d.%n" ?ptname ?newbedno)
        )
    )
)
""")

In [27]:
env.build("""
(defrule R2 "IF there is a patient in an ICU bed who is not very ill and not infectious and there is a Unit 1 bed free THEN transfer to Unit 1 bed"
    (patient (health-status $?status)(patient-name ?ptname))
    (test (not (member$ infectious $?status)))
    (test (member$ not-very-ill $?status))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ICU))
    ?newbed-fact <- (bed (patient-name nil) (unit Unit1)(bed-number ?bedno))
    =>
    (modify ?oldbed-fact (patient-name nil))
    (modify ?newbed-fact (patient-name ?ptname))
    (bind ?*log* 
        (str-cat ?*log*
          (format nil "R2 Fired. Patient %s has been moved to Unit1 bed %d.%n" ?ptname ?bedno)
        )
     )
)
""")

env.build("""
(defrule R3 "If patient is ready for discharge, then free the bed"
    ?patient-fact <- (patient (health-status $? free2go $?)(patient-name ?ptname))
    ?newbed-fact <- (bed (patient-name ?ptname)(unit ?unitname)(bed-number ?bedno))
    =>
    (retract ?patient-fact)
    (modify ?newbed-fact (patient-name nil))
    (bind ?*log* 
        (str-cat ?*log*
	     (format nil "R3 Fired. Patient %s has been discharged, and %s bed %d is now free.%n" ?ptname  ?unitname  ?bedno)
        )
    )
)
""")

env.build("""
(defrule R4 "IF a patient is infectious and not very ill and there is a Unit 2 bed free THEN admit or transfer only to unit 2"
    (patient (health-status $?status)(patient-name ?ptname))
    (test (member$ infectious $?status))
    (test (member$ not-very-ill $?status))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ER|ICU)) ; why (unit ER|ICU)?
    ?newbed-fact <- (bed (patient-name nil) (unit Unit2)(bed-number ?bedno))
    =>
    (modify ?oldbed-fact (patient-name nil))
    (modify ?newbed-fact (patient-name ?ptname))
    (bind ?*log* 
        (str-cat ?*log*
           (format nil "R4 Fired. Patient %s has been moved to Unit2 bed %d.%n" ?ptname ?bedno)
        )
     )
)    
""")

In [28]:
env.run()

3

In [29]:
v = env.find_global('log')
print(v.value)
for fact in env.facts():
    print(fact)

INFO:
R3 Fired. Patient Frank has been discharged, and Unit2 bed 2 is now free.
R4 Fired. Patient Barry has been moved to Unit2 bed 2.
R1 Fired. Patient Ann has been moved to ICU bed 1.

(initial-fact)
(bed (unit ICU) (bed-number 2) (patient-name Cleo))
(bed (unit Unit1) (bed-number 1) (patient-name nil))
(bed (unit Unit1) (bed-number 2) (patient-name Darren))
(bed (unit Unit2) (bed-number 1) (patient-name Eva))
(patient (patient-name Ann) (health-status very-ill))
(patient (patient-name Barry) (health-status not-very-ill infectious))
f-10    (patient (patient-name Cleo) (health-status infectious very-ill))
f-11    (patient (patient-name Darren) (health-status not-very-ill))
f-12    (patient (patient-name Eva) (health-status infectious not-very-ill))
f-16    (bed (unit Unit2) (bed-number 2) (patient-name Barry))
f-17    (bed (unit ICU) (bed-number 1) (patient-name Ann))


The Magic Squares program from Riggs 2000

In [86]:
# Sets up a Magic Square per figure 1
env.clear()
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')
env.build("""
(deffacts init
 (element 1 1 8)
 (element 1 2 1)
 (element 1 3 6)
 (element 2 1 3)
 (element 2 2 5)
 (element 2 3 7)
 (element 3 1 4)
 (element 3 2 9)
 (element 3 3 2)
 (sum row 1 0)
 (sum row 2 0)
 (sum row 3 0)
 (sum col 1 0)
 (sum col 2 0)
 (sum col 3 0)
 (sum diag 1 0)
 (sum diag 2 0)
 (size 3)
)"""
)

In [87]:
env.reset()
for fact in env.facts():
    print(fact)

(initial-fact)
(element 1 1 8)
(element 1 2 1)
(element 1 3 6)
(element 2 1 3)
(element 2 2 5)
(element 2 3 7)
(element 3 1 4)
(element 3 2 9)
(element 3 3 2)
f-10    (sum row 1 0)
f-11    (sum row 2 0)
f-12    (sum row 3 0)
f-13    (sum col 1 0)
f-14    (sum col 2 0)
f-15    (sum col 3 0)
f-16    (sum diag 1 0)
f-17    (sum diag 2 0)
f-18    (size 3)


In [88]:
env.build("""
(defrule make-addends
 (declare (salience 100)) ;; salience > default
 (size ?size)
 (element ?r ?c ?v)
  =>
 (assert 
   (addend row ?r ?v)
   (addend col ?c ?v))
 (if (= ?r ?c) ;; on diagl?
   then (assert (addend diag 1 ?v)))
 (if (= (+ ?r ?c) (+ 1 ?size)) ;; on diag2?
   then (assert (addend diag 2 ?v)))) 
""")

In [89]:
env.build("""
(defrule addem
 (declare (salience 100))
 ?fl <- (addend ?type ?inx ?val)
 ?f2 <- (sum ?type ?inx ?sum)
 =>
 (retract ?fl ?f2) 
 (assert (sum ?type ?inx (+ ?val ?sum)))
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "type: %s; val: %d; sum: %d%n" ?type ?val ?sum)
   )
 )
)
""")

In [90]:
env.build("""
(defrule not-magic
 (sum ? ? ?s)
 (sum ? ? ?r&~?s) ;; i.e. ?s <> ?r
 =>
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "Not Magic.%n")
    )
 )
 (halt)
)
""")

env.build("""
(defrule magic
 (sum row 1 ?val)
 (not (sum ? ? ?v2&~?val)) ;; i.e. not ?v2 <> val
 =>
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "Magic!!%n")
   )
  )
) 
""")

In [91]:
env.run()
v = env.find_global('log')
print(v.value)
for fact in env.facts():
    print(fact)

INFO:
type: diag; val: 2; sum: 0
type: col; val: 2; sum: 0
type: row; val: 2; sum: 0
type: col; val: 9; sum: 0
type: row; val: 9; sum: 2
type: diag; val: 4; sum: 0
type: col; val: 4; sum: 0
type: row; val: 4; sum: 11
type: col; val: 7; sum: 2
type: row; val: 7; sum: 0
type: diag; val: 5; sum: 4
type: diag; val: 5; sum: 2
type: col; val: 5; sum: 9
type: row; val: 5; sum: 7
type: col; val: 3; sum: 4
type: row; val: 3; sum: 12
type: diag; val: 6; sum: 9
type: col; val: 6; sum: 9
type: row; val: 6; sum: 0
type: col; val: 1; sum: 14
type: row; val: 1; sum: 6
type: diag; val: 8; sum: 7
type: col; val: 8; sum: 7
type: row; val: 8; sum: 7
Magic!!

(initial-fact)
(element 1 1 8)
(element 1 2 1)
(element 1 3 6)
(element 2 1 3)
(element 2 2 5)
(element 2 3 7)
(element 3 1 4)
(element 3 2 9)
(element 3 3 2)
f-18    (size 3)
f-34    (sum row 3 15)
f-50    (sum row 2 15)
f-54    (sum diag 2 15)
f-55    (sum col 3 15)
f-59    (sum col 2 15)
f-64    (sum diag 1 15)
f-65    (sum col 1 15)
f-66    (sum 

Dining Philosophers

In [104]:
env.clear()
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')
env.build("""
(deffacts init 
 (p 1 uses 5 1)
 (p 2 uses 1 2)
 (p 3 uses 2 3)
 (p 4 uses 3 4)
 (p 5 uses 4 5)
 (fork 1 avail)
 (fork 2 avail)
 (fork 3 avail)
 (fork 4 avail)
 (fork 5 avail)
 (p 1 thinking)
 (p 2 thinking)
 (p 3 thinking)
 (p 4 thinking)
 (p 5 thinking)
)"""
)
env.reset()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-10    (fork 5 avail)
f-11    (p 1 thinking)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)


In [105]:
env.build("""
(defrule eat ;; hungry -> eat
  ?a<-(active ?n) ;; serialization
  ?p<-(p ?n hungry)
      (p ?n uses ?left ?right)
  ?f<-(fork ?right avail)
 =>
 (retract ?a ?p ?f)
 (assert (p ?n eats)   ;; eat
    (fork ?right ?n))  ;; 2nd fork taken
)
""")

env.build("""
(defrule hunger  ;; thinking -> hungry
  ?a<-(active ?n) ;; serialization
  ?p<-(p ?n thinking)
      (p ?n uses ?left ?right)
  ?f<-(fork ?left avail)
 =>
 (retract ?a ?p ?f)
 (assert (p ?n hungry)     ;; hungry
         (fork ?left ?n))  ;; 1st fork taken
)
""")

env.build("""
(defrule think ;; eat -> thinking
 ?a<-(active ?n) ;; serialization
 ?p<-(p ?n eats)
     (p ?n uses ?left ?right)
 ?f1<-(fork ?left ?n)
 ?f2<-(fork ?right ?n)
 =>
 (retract ?a ?p ?f1 ?f2 )
 (assert (p ?n thinking)         ;; thinking
           (fork ?left avail)    ;; fork 1 returned
           (fork ?right avail))  ;; fork 2 returned
)
""")

env.build("""
(defrule dont-eat   ;; hungry -> not eat
 ?a<-(active ?n)    ;; serialization
 ?p<-(p ?n hungry)
     (p ?n uses ?left ?right)
 ?f<- (fork ?left ?n)
      (not (fork ?right avail)) ;; can't get right
  =>
 (retract ?a ?p ?f)
 (assert (p ?n thinking)
         (fork ?left down))  ;; release left
)
""")


In [106]:
env.run()
v = env.find_global('log')
print(v.value)
for fact in env.facts():
    print(fact)

INFO:

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-10    (fork 5 avail)
f-11    (p 1 thinking)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)


In [107]:
env.assert_string('(active 1)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)


In [108]:
env.assert_string('(active 2)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)


In [109]:
env.assert_string('(active 3)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 3 avail)
(fork 4 avail)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)


In [110]:
env.assert_string('(active 4)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 4 avail)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)


In [111]:
env.assert_string('(active 5)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)
f-29    (p 5 hungry)
f-30    (fork 4 5)


In [112]:
env.assert_string('(active 2)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)
f-29    (p 5 hungry)
f-30    (fork 4 5)
f-32    (p 2 thinking)
f-33    (fork 1 down)


Testing the application of rules to data extracted using a FHIR server

In [131]:
# Using https://github.com/smart-on-fhir/client-py for FHIR STU3 
# and our own synthetic data on AWS
from fhirclient import client
settings = {
    'app_id':'no matter',
    'api_base': 'http://localhost:8080/omoponfhir-stu3/fhir/'
}
syn_omop = client.FHIRClient(settings=settings)

In [132]:
import fhirclient.models.patient as p
### Does not work b/c auth token required
patient = p.Patient.read('1495', syn_omop.server)

## Works - requests URL https://syntheticmass.mitre.org/v1/fhir/Patient/c8a021dc-b91e-4fe7-b6ec-81c948013fc4?apikey=Er32s112u0OOkZohfdHmxJUDGAJHKai2
## patient = p.Patient.read_from('Patient/c8a021dc-b91e-4fe7-b6ec-81c948013fc4?apikey=Er32s112u0OOkZohfdHmxJUDGAJHKai2', smart.server)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /omoponfhir-stu3/fhir/Patient/1495 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f53eea6db70>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [128]:
print(patient.birthDate.isostring)
print(smart.human_name(patient.name[0]))

1952-07-03
Mr. Lionel365 Beer512


In [ ]:
import fhirclient.models.medicationrequest as mr
search = mr.MedicationRequest.where(struct={'patient': 'c8a021dc-b91e-4fe7-b6ec-81c948013fc4?apikey=Er32s112u0OOkZohfdHmxJUDGAJHKai2'})